# Introduction
The following code for NSE has been picked up from [Ewald's examples](https://github.com/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb) given below

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

from ib_insync import *
util.startLoop()

ib=IB()
ib.connect('127.0.0.1', 4004, clientId=12)

<IB connected to 127.0.0.1:4004 clientId=12>

In [2]:
# instantiate stock contract class
scrip = Stock(symbol='RELIANCE', exchange='NSE' )
ib.qualifyContracts(scrip)

[Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE')]

In [3]:
# get the underlying's option chain with security definition parameter
chains = ib.reqSecDefOptParams(underlyingSymbol=scrip.symbol, futFopExchange = '', 
                               underlyingSecType=scrip.secType, underlyingConId=scrip.conId)

# put that in a dataframe
chains_df = util.df(chains)
chains_df.head()

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,NSE,44652000,RELIANCE,1,"{20180726, 20180830, 20180927}","{640.0, 1280.0, 900.0, 1400.0, 520.0, 1160.0, ..."


In [17]:
# make arrays for strikes and expiries
strikes = np.array(list(*chains_df.strikes))
expiries = np.array(list(*chains_df.expirations))

# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

dte = list(map(get_dte, expiries))

def get_stdev(contract, dte):
    '''Gets the Standard Deviation
    Args:
       (contract) = object: the qualified stock
       (dte)      = int: days to expiry
    Returns:
       standard deviation in days (int)
    '''
    sd_days = str(dte)+' D'
    bars = ib.reqHistoricalData(contract=scrip, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    sd = np.std([b.close for b in bars], ddof=1)
    return sd

stdev = list(map(lambda x, y: get_stdev(x, y), np.repeat(scrip, len(dte)), dte))

# equalize lengths of array for strikes+expiries+dte
a_strikes = np.repeat(strikes, len(expiries))
a_expiries = np.tile(expiries, len(strikes))   # tile to repeat expiries in blocks
a_dte = np.tile(dte, len(strikes))
a_stdev = np.tile(stdev, len(strikes))

# create a dataframe of Strike and Expiry
strikes_df = pd.DataFrame({'Strike': a_strikes, 'Expiry': a_expiries, 'DTE': a_dte, 'Stdev': a_stdev})

# insert a key and merge strikes and chains
strikes_df['key'] = 0
chains_df['key'] = 0

raw_chain = pd.merge(strikes_df, chains_df, on='key')
raw_chain = raw_chain.drop(['expirations', 'strikes', 'key'], axis=1)

raw_chain.head()

,Strike,Expiry,DTE,Stdev,exchange,underlyingConId,tradingClass,multiplier
0,640.0,20180726,19,23.741766,NSE,44652000,RELIANCE,1
1,640.0,20180830,54,30.505559,NSE,44652000,RELIANCE,1
2,640.0,20180927,82,36.769225,NSE,44652000,RELIANCE,1
3,1280.0,20180726,19,23.741766,NSE,44652000,RELIANCE,1
4,1280.0,20180830,54,30.505559,NSE,44652000,RELIANCE,1


23.176121445642234

[23.697995078914033, 30.518694895427554, 36.78629627007997]

In [14]:
np.repeat(scrip, len(dte))

array([Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE'),
       Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE'),
       Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE')],
      dtype=object)

# Original code of Ewald

In [2]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

In [3]:
# Get the ticker
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2018, 7, 4, 14, 52, 15, 23498, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=2726.71, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

In [4]:
spxValue = ticker.marketPrice()

In [5]:
spxValue

2726.71

In [6]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"{20180920, 20190314, 20190117, 20191219, 20200...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."
1,CBOE,416904,SPXW,100,"{20180711, 20190621, 20180709, 20180727, 20180...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."
2,SMART,416904,SPXW,100,"{20180711, 20190621, 20180709, 20180727, 20180...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."
3,CBOE,416904,SPX,100,"{20180920, 20190314, 20190117, 20191219, 20200...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."


In [2]:
scrip = Stock('INTC', 'SMART', 'USD')
ib.qualifyContracts(scrip)

[Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS')]

In [3]:
scrip_chain = ib.reqSecDefOptParams(scrip.symbol, '', scrip.secType, scrip.conId)
util.df(scrip_chain)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
1,EDGX,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
2,CBOE2,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
3,ISE,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
4,NASDAQBX,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
5,MIAX,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
6,CBOE,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
7,MERCURY,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
8,NASDAQOM,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."
9,BATS,270639,INTC,100,"{20180824, 20180817, 20200117, 20180720, 20190...","{61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54...."


In [20]:
scrip_chain

[OptionChain(exchange='SMART', underlyingConId=270639, tradingClass='INTC', multiplier='100', expirations={'20180824', '20180921', '20180720', '20190118', '20181019', '20190621', '20180727', '20180803', '20180713', '20180817', '20200117', '20180810', '20180706'}, strikes={61.5, 51.5, 56.5, 52.5, 58.5, 53.5, 57.5, 54.5, 18.0, 20.0, 55.5, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 37.5, 38.0, 39.0, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 65.0, 70.0, 75.0, 59.5, 80.0, 85.0, 60.5, 48.5, 49.5, 50.5, 62.5}),
 OptionChain(exchange='EDGX', underlyingConId=270639, tradingClass='INTC', multiplier='100', expirations={'20180824', '20180921', '20180720', '20190118', '20181019', '20190621', '20180727', '20180803', '20180713', '20180817', '20200117', '20180810', '20180706'}, strikes={61.5, 51.5, 56.5, 52.

In [157]:
help(Contract.right)

Help on member descriptor ib_insync.contract.Contract.right:

right

